In [30]:
import fmpsdk as fmp
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv("./scripts/.env")
fmp_key = os.environ.get("FMP_KEY")
import plotly.express as px

In [12]:
help(fmp.cash_flow_statement)

Help on function cash_flow_statement in module fmpsdk.company_valuation:

cash_flow_statement(apikey: str, symbol: str, period: str = 'annual', limit: int = 10, download: bool = False, filename: str = 'cash_flow_statement.csv') -> Optional[List[Dict]]
    Query FMP /cash-flow-statement/ API.
    
    Display or download company's cash flow statement.
    :param apikey: Your API key.
    :param symbol: Company ticker.
    :param period: 'quarter' or 'annual'.
    :param limit: Number of rows to return.
    :param download: True/False
    :param filename: Name of saved file.
    :return: A list of dictionaries.



In [64]:
income = pd.DataFrame(fmp.income_statement(symbol="AMD",apikey=fmp_key,period="quarter"))[["date","revenue"]]

In [65]:
income

,date,revenue
0,2023-12-30,6168000000
1,2023-09-30,5800000000
2,2023-07-01,5359000000
3,2023-04-01,5353000000
4,2022-12-31,5599000000
5,2022-09-24,5565000000
6,2022-06-25,6550000000
7,2022-03-26,5887000000
8,2021-12-25,4826000000
9,2021-09-25,4313000000


In [62]:
balance = pd.DataFrame(fmp.cash_flow_statement(symbol="AMD",apikey=fmp_key,period="quarter"))[['date','inventory']]

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [51]:
data = pd.concat([income.date,revenue_by_inventory],axis=1)

In [56]:
data

,date,0
0,2023-12-30,241.276596
1,2022-12-31,193.450820
2,2021-12-25,-49.500000
3,2020-12-26,-21.040948
4,2019-12-28,-16.660891
5,2018-12-29,-8.955740
6,2017-12-30,-21.487903
7,2016-12-31,-164.307692
8,2015-12-26,-76.750000
9,2014-12-27,-39.898551


In [85]:
def profitabilty(symbol, duration="quarter"):
    load_dotenv("./scripts/.env")
    fmp_key = os.environ.get("FMP_KEY")

    if isinstance(symbol, str):
            symbols = [symbol]
    elif isinstance(symbol, list):
            symbols = symbol
    else:
        raise ValueError("Symbol must be a string or a list of strings")
    # Retrieve company profile

    revenue_data = {}
    for symbol in symbols:
        revenue = pd.DataFrame(fmp.income_statement(symbol=symbol, apikey=fmp_key,period="quarter"))[["date","revenue"]]
        inventory = pd.DataFrame(fmp.cash_flow_statement(symbol=symbol ,apikey=fmp_key,period="quarter"))[['date','inventory']]
        revenue[symbol] = revenue["revenue"] / inventory["inventory"]
        revenue_data[symbol] = revenue
    # data = pd.DataFrame(revenue_data)
    combined_data = pd.DataFrame()
    for i, x in revenue_data.items():
        date = x["date"]
        combined_data = pd.concat([combined_data, x[[i]]],axis=1,join="outer")
    combined_data["date"] =  date
    return combined_data

In [86]:
prof = profitabilty(["AMD","NVDA"])

In [95]:
prof

{'AMD':          date     revenue         AMD
 0  2023-12-30  6168000000   65.617021
 1  2023-09-30  5800000000   47.540984
 2  2023-07-01  5359000000  -16.141566
 3  2023-04-01  5353000000  -11.536638
 4  2022-12-31  5599000000  -13.858911
 5  2022-09-24  5565000000   -7.697095
 6  2022-06-25  6550000000  -26.411290
 7  2022-03-26  5887000000 -226.423077
 8  2021-12-25  4826000000  -92.807692
 9  2021-09-25  4313000000  -31.253623,
 'NVDA':          date      revenue       NVDA
 0  2024-01-28  22103000000 -43.942346
 1  2023-10-29  18120000000 -39.736842
 2  2023-07-30  13507000000  45.786441
 3  2023-04-30   7192000000  12.706714
 4  2023-01-29   6051000000  -8.570822
 5  2022-10-30   5931000000 -10.534636
 6  2022-07-31   6704000000  -9.246897
 7  2022-05-01   8288000000 -14.800000
 8  2022-01-30   7643000000 -20.435829
 9  2021-10-31   7103000000 -60.194915}

In [87]:
combined_data = pd.concat(prof.values())

In [88]:
combined_data

,date,revenue,AMD,NVDA
0,2023-12-30,6168000000,65.617021,NaN
1,2023-09-30,5800000000,47.540984,NaN
2,2023-07-01,5359000000,-16.141566,NaN
3,2023-04-01,5353000000,-11.536638,NaN
4,2022-12-31,5599000000,-13.858911,NaN
5,2022-09-24,5565000000,-7.697095,NaN
6,2022-06-25,6550000000,-26.411290,NaN
7,2022-03-26,5887000000,-226.423077,NaN
8,2021-12-25,4826000000,-92.807692,NaN
9,2021-09-25,4313000000,-31.253623,NaN


In [96]:
combined_data = pd.DataFrame()
for i, x in prof.items():
    date = x["date"]
    combined_data = pd.concat([combined_data, x[[i]]],axis=1,join="outer")
    
combined_data["date"] =  date


In [103]:
combined_data

,AMD,NVDA,date
0,65.617021,-43.942346,2024-01-28
1,47.540984,-39.736842,2023-10-29
2,-16.141566,45.786441,2023-07-30
3,-11.536638,12.706714,2023-04-30
4,-13.858911,-8.570822,2023-01-29
5,-7.697095,-10.534636,2022-10-30
6,-26.411290,-9.246897,2022-07-31
7,-226.423077,-14.800000,2022-05-01
8,-92.807692,-20.435829,2022-01-30
9,-31.253623,-60.194915,2021-10-31


In [112]:
revByinv_fig = px.line(combined_data, x="date", y=combined_data.columns,labels={'x': 'Date', 'y': 'Revenue/Inventory'})
fig.show()

In [116]:
fmp.insider_trading(fmp_key,"TKO")

[{'symbol': 'TKO',
  'filingDate': '2024-04-11 19:00:07',
  'transactionDate': '2024-04-09',
  'reportingCik': '0001531362',
  'transactionType': 'P-Purchase',
  'securitiesOwned': 1642970,
  'companyCik': '0001973266',
  'reportingName': 'Whitesell Patrick',
  'typeOfOwner': '10 percent owner',
  'acquistionOrDisposition': 'A',
  'formType': '4',
  'securitiesTransacted': 1642970,
  'price': 89.01,
  'securityName': 'Class A Common Stock',
  'link': 'https://www.sec.gov/Archives/edgar/data/1766363/000095017024043816/0000950170-24-043816-index.htm'},
 {'symbol': 'TKO',
  'filingDate': '2023-11-14 21:39:47',
  'transactionDate': '2023-11-14',
  'reportingCik': '0001233831',
  'transactionType': 'S-Sale',
  'securitiesOwned': 20435207,
  'companyCik': '0001973266',
  'reportingName': 'MCMAHON VINCENT K',
  'typeOfOwner': 'director, 10 percent owner: ',
  'acquistionOrDisposition': 'D',
  'formType': '4',
  'securitiesTransacted': 8400000,
  'price': 76.41,
  'securityName': 'Class A Comm